In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensor.operation.kruskal import kruskal
from tensor.operation.khatri_rao import khatri_rao
from tensor.operation.matricize import matricize

## CP decomposition

Algorithm: CP decomposition ALS

$\text{Input:} \quad \text{Tensor } X \in \mathbb{R}^{J_1 \times J_2 \times \cdots \times J_N}, \text{ rank } R$

$\text{Output:} \quad \text{Factor matrices } U_1, U_2, \cdots, U_N \\
$

\begin{align*}
&\text{Initialize } U_1, U_2, \cdots, U_N \text{ with random matrices} \\
&\text{while } \text{not converged } \text{do} \\
& \quad \quad \text{for } n = 1, 2, \cdots, N \text{ do} \\
& \quad \quad \quad \text{Compute } \mathbf{A}^{(n)^T} = \left( \mathbf{A}^{(N)} \odot \mathbf{A}^{(N-1)} \odot \cdots \odot \mathbf{A}^{(n+1)} \odot \mathbf{A}^{(n-1)} \cdot \mathbf{A}^{(1)} \right )^{\dag} X^T_{(n)} \\
& \quad \quad \text{end for} \\
&\text{end while} \\
\end{align*}

In [6]:
def cpDecomposition(X: np.ndarray, rank: int, maxIter: int = 5, tol: float = 1e-6):
    """cpDecomposition performs CP decomposition of a tensor X using alternating least squares.

    Args:
        X (np.ndarray): Tensor to be decomposed.
        rank (int): Rank of the decomposition.
        maxIter (int, optional): Maximum number of iterations. Defaults to 1000.
        tol (float, optional): Tolerance for the stopping criterion. Defaults to 1e-6.

    Returns:
        np.ndarray: Factor matrices of the decomposition.

    """
    # Initialize factor matrices
    U = [np.random.rand(X.shape[i], rank) for i in range(X.ndim)]

    # Iterate until convergence
    for itr in range(maxIter):

        for i in range(X.ndim):

            khatriRaoProd = np.ones((1, rank))
            for j in range(X.ndim, 0, -1):
                if j != (i + 1):
                    khatriRaoProd = khatri_rao(khatriRaoProd, U[j - 1])

            U[i] = matricize(X, i) @ np.linalg.pinv(khatriRaoProd).T
        print("Iteration ", itr+1, " completed. loss =",
              np.linalg.norm(X - kruskal(*U)))

        # Check for convergence
        if np.linalg.norm(X - kruskal(*U)) < tol:
            break

    return np.array(U)


In [7]:
X = np.array([[[1, -1], [0, 0]], [[0, 0], [1, 1]]])
ans = cpDecomposition(X, 3, maxIter=20)

Iteration  1  completed. loss = 0.24952368068224098
Iteration  2  completed. loss = 0.00043596515395754134
Iteration  3  completed. loss = 1.0781354814328967e-09


/home/ganesh/anaconda3/envs/CS3390/lib/python3.10/site-packages/numpy/core/fromnumeric.py:86: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


In [8]:
print("A = \n",ans[0]) 
print("B = \n",ans[1])
print("C = \n",ans[2])

A = 
 [[-7.22404010e-01 -9.30398957e-01  2.58363737e+00]
 [-1.45917348e-03  4.87359604e+00 -3.16038679e-01]]
B = 
 [[ 1.44090841  0.04683864  0.84775412]
 [-0.00428549  0.36126171  0.15269187]]
C = 
 [[0.056609   0.58154203 0.49503423]
 [1.94604955 0.58111904 0.4798464 ]]
